In [16]:
! pip install nytimesarticle

In [17]:
from nytimesarticle import articleAPI
api = articleAPI('c07399afd36e41d8a8e0b4b65ad47e27')

ImportError: No module named nytimesarticle

In [10]:
articles_0 = api.search( q = 'Brexit', 
     fq = {'headline':'Brexit', 'source':['AP', 'The New York Times']}, 
     begin_date = "20130101", sort = "oldest" )
articles_0

{u'copyright': u'Copyright (c) 2017 The New York Times Company. All Rights Reserved.',
 u'response': {u'docs': [{u'_id': u'55ffd71638f0d806a3a1cfa9',
    u'blog': {},
    u'byline': {u'original': u'By MATTHEW J. GOODWIN'},
    u'document_type': u'article',
    u'headline': {u'kicker': u'Op-Ed Contributor',
     u'main': u'Why a \u2018Brexit\u2019 Looms Large',
     u'print_headline': u"Why a 'Brexit' Looms Large"},
    u'keywords': [{u'isMajor': None,
      u'name': u'subject',
      u'rank': 1,
      u'value': u'Referendums'},
     {u'isMajor': None,
      u'name': u'organizations',
      u'rank': 2,
      u'value': u'European Union'},
     {u'isMajor': None,
      u'name': u'glocations',
      u'rank': 3,
      u'value': u'Great Britain'},
     {u'isMajor': None,
      u'name': u'subject',
      u'rank': 4,
      u'value': u'Great Britain Withdrawal from EU (Brexit)'}],
    u'multimedia': [{u'height': 126,
      u'legacy': {u'wide': u'images/2015/09/21/opinion/21goodwinart/21goodwina

In [11]:
def parse_articles(articles):
    '''
    This function takes in a response to the NYT api and parses
    the articles into a list of dictionaries
    '''
    news = []
    for i in articles['response']['docs']:
        dic = {}
        dic['id'] = i['_id']
        dic['headline'] = i['headline']['main'].encode("utf8")
        dic['date'] = i['pub_date'][0:10] # cutting time of day.
        if i['snippet'] is not None:
            dic['snippet'] = i['snippet'].encode("utf8")
        dic['type'] = i['type_of_material']
        
        news.append(dic)
    return(news) 

In [12]:
def get_articles(date,query):
    '''
    This function accepts a year in string format (e.g.'1980')
    and a query (e.g.'Brexit') and it will 
    return a list of parsed articles (in dictionaries)
    for that year.
    '''
    import time
    all_articles = []
    for i in range(0,100): #NYT limits pager to first 100 pages. 
        articles = api.search(q = query,
                              fq = {'source':['The New York Times']}, 
                              begin_date = date*10000 + 101, 
                              end_date = date*10000 + 1231, 
                              sort='oldest', 
                              page = str(i))
        articles = parse_articles(articles)
        all_articles = all_articles + articles
        time.sleep(1.1)#The Article Search API is rate limited to 1 call per second.
    return(all_articles)

In [13]:
Brexit_all = []
for i in range(2013,2018):
    print 'Processing' + str(i) + '...'
    Brexit_year =  get_articles(i,'Brexit')
    Brexit_all = Brexit_all + Brexit_year

Processing2013...
Processing2014...
Processing2015...
Processing2016...
Processing2017...


In [14]:
Brexit_all

[{'date': u'2013-01-01',
  'headline': 'What to Watch for in Europe in 2013',
  'id': u'50e2ea6100315214fbb81efa',
  'locations': [u'Europe'],
  'snippet': "The economy, elections, and how to deal with a new Islamist threat: as politicians make their New Year resolutions, these are a few items on Europe's agenda for 2013.",
  'subjects': [u'Elections', u'New Year'],
  'type': u'Blog',
  'url': u'https://rendezvous.blogs.nytimes.com/2013/01/01/what-to-watch-for-in-europe-in-2013/'},
 {'date': u'2014-06-25',
  'headline': 'David Cameron\xe2\x80\x99s Losing Battle',
  'id': u'53aaa68638f0d86af25be96a',
  'locations': [u'Great Britain', u'Luxembourg', u'Europe'],
  'snippet': 'The British prime minister\xe2\x80\x99s case against Jean-Claude Juncker could well boomerang and feed anti-E.U. sentiments in Britain.',
  'subjects': [u'Elections', u'Editorials'],
  'type': u'Editorial',
  'url': u'https://www.nytimes.com/2014/06/26/opinion/mr-camerons-losing-battle.html'},
 {'date': u'2014-06-30'

In [16]:
import csv
keys = Brexit_all[0].keys()
with open('Brexit-mentions.csv', 'wb') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(Brexit_all)